在训练参数化机器学习模型时， 权重衰减（weight decay）是最广泛使用的正则化的技术之一， 它通常也被称为$L_2$正则化。        
一种简单的方法是通过线性函数$f(x)=w^Tx$  中的权重向量的某个范数来度量其复杂性， 例如$||w^2||$。 要保证权重向量比较小， 最常用方法是将其范数作为惩罚项加到最小化损失的问题中。     
$$
L(w, b) = \frac{1}{n} \sum_{i=1}^n \frac{1}{2} (w^Tx^{(i)} + b -y^{(i)}) ^ 2   
$$

### 高维线性回归

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

生成一些数据，生成公式如下：
$$
y = 0.05 + \sum^d_{i=1}0.01x_i + \epsilon  \quad  where \epsilon ~ \Nu(0, 0.01^2)
$$

In [ ]:
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = torch.ones((num_inputs, 1)) * 0.01, 0.05
train_data = d2l.synthetic_data(true_w, true_b, n_train)
train_iter = d2l.load_array(train_data, batch_size)
test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)

初始化模型参数

In [ ]:
def init_params():
    w = torch.normal(0, 1, size=(num_inputs, 1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    return [w, b]

定义$L_2$范数惩罚

In [ ]:
def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

定义训练代码实现

In [ ]:
def train(lambd):
    w, b = init_params()
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    num_epochs, lr = 100, 0.03
    animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log', xlim=[5, num_epochs], legend=['train', 'test'])
    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y) + lambd * l2_penalty(w)
            l.sum().backward()
            d2l.sgd([w, b], lr, batch_size)
        if (epoch + 1) % 5 == 0:
            animator.add(epoch+1, (d2l.evaluate_loss(net, train_data, loss),
                d2l.evaluate_loss(net, test_data, loss)))
    print('w的L2范数是: ', torch.norm(w).item())

训练

In [ ]:
train(lambd=0)